# Cuaderno 18: Caminos más cortos con ventanas de tiempo
# (Shortest Path Problems with Time Windows - SPPTW)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo dirigido $D=(V,A)$; 
* un vector de costos $c \in \ZZ^{A}$  asociados a los arcos de $D$; 
* un vector de tiempos de tránsito $t \in \ZZ^{A}_{+}$  asociados a los arcos de $D$; 
* un nodo de salida $r$;
* un nodo de llegada $s$, y
* una ventana de tiempo $[a_i;b_i]$ para cada nodo $i$ in $V$.

El *problema del camino más corto con ventanas de tiempo* consiste en encontrar un camino $P$ desde $r$ hasta $s$, conjuntamente con un tiempo de partida $T_r$, de tal forma que cada nodo de $P$ sea visitado dentro de su ventana de tiempo, y que el costo del camino sea mínimo. El costo de un camino se calcula sumando los costos de sus arcos. Por otra parte, los tiempos de visita de los nodos se definen de la siguiente manera:
* el tiempo de visita de $r$ es el tiempo de partida $T_r$ del camino,
* si $(i,j)$ es un arco de $P$, entonces para los tiempos de visita $T_i$ y $T_j$ sus nodos extremos, debe cumplirse que $T_j := \max\{T_i + t_{ij}, a_j\}$.

Para formular este problema, utilizaremos la siguientes variables de decisión: 
* variables binarias $x_{ij}$, para indicar si los arcos de $A$ son seleccionados o no dentro de $P$; y, 
* variables no negativas $T_i$ para indicar el tiempo de visita de un nodo $i$ del camino; si el nodo $i$ no pertenece a $P$, el valor de $T_i$ es irrelevante para el modelo.

De esta manera, se puede formular al problema como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{(r, i) \in A} x_{ri} = 1,\\
&\sum_{(i, s) \in A} x_{is} = 1,\\
& \sum_{(j, i) \in A} x_{ji} - \sum_{(i, j) \in A} x_{ij} = 0, \quad \forall i \in V \setminus \tabulatedset{r,s},\\
&T_j \geq T_i + t_{ij} - M(1 -  x_{ij}), \quad \forall (i,j) \in A, \\ 
&a_i \leq T_i \leq b_i, \quad \forall i \in V,\\ 
&T_i \geq 0, \quad \forall i \in V,\\
& x_{ij} \in \tabulatedset{0, 1}, \quad \forall (i, j) \in A.
\end{align*}

El modelo es similar al modelo empleado para el problema de caminos más cortos con duración acotada. La función objetivo mide el costo total de los arcos seleccionados, mientras que las primeras tres familias de restricciones son restricciones de grado: seleccionar exactamente un arco saliente de $r$, exactamente un arco entrante a $s$, y un número de arcos entrantes igual al número de arcos salientes para los demás nodos. 

En la cuarta familia de restricciones se emplea la técnica de la $M$-grande descrita en el Cuaderno 17 para especificar que, si el arco $(i,j)$ es seleccionado en la solución, entonces debe cumplirse que $T_j \geq T_i + t_{ij}$. 

La quinta familia de restricciones requiere que los valores del tiempo de visita en cada nodo estén dentro de las ventanas de tiempo.

Notar que, en conjunto, la cuarta y quinta familias de restricciones implican que debe cumplirse $T_j \geq \max\{T_i + t_{ij}, a_j\}$ para todo nodo $j$ visitado por el camino. Es posible demostrar, además, que dada una solución con estas características, puede obtenerse una solución factible al problema con el mismo costo, al ajustar $T_j:= \max\{T_i + t_{ij}, a_j\}$ para todo nodo $j$ visitado por el camino.

Al igual que el modelo para caminos más cortos don duración acotada, este modelo funciona aún si el grafo tiene circuitos de costo negativo, siempre y cuando los tiempos de tránsito sean positivos.

Vamos a implementar este modelo usando la interfaz Python de Gurobi.



Definimos primero los conjuntos y parámetros del modelo:

In [ ]:
from gurobipy import *

# Nodos y ventanas de tiempo
V , a, b = multidict({
    1 : (0, 1),
    2 : (2, 4),
    3 : (1, 3),
    4 : (4, 6),
    5 : (5, 6),
    6 : (5, 7),
    7 : (6, 7),
    8 : (7, 9),
})

# Arcos, costos y tiempos de tránsito 
A, c, t = multidict({
  (1, 2):  (3, 2), 
  (1, 3):  (20,1),
  (2, 3):  (10,2),
  (2, 4):  (5,2),
  (4, 3):  (2,3),
  (3, 5):  (2,3),
  (4, 5):  (5,1),
  (4, 6):  (10,1),
  (5, 6):  (2,3),
  (5, 7):  (-3,1),
  (6, 8):  (1, 2),  
  (7, 8):  (-3,1),  
  (8, 5):  (-3,1)})

# Nodo de salida
r = 1

# Nodo de llegada
s = 8

# Constante suficientemente grande:
M = quicksum([t[i,j] for (i,j) in A]) + quicksum(b[i] for i in V)

# --- los valores a partir de aqui se calculan automaticamente ---
# nodos internos: Vi := V \ {s, t}
Vi = tuplelist([i for i in V if i!=r and i!=s])

En la siguiente celda empleamos los módulos `networkx` y `matplotlib` para representar graficamente esta instancia del problema:

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
D = nx.DiGraph()
D.add_nodes_from(V)
node_labels= {i : '{}\n[{};{}]'.format(i, a[i], b[i]) for i in V}
D.add_edges_from(A)
edge_labels = {(i,j) : str(c[i,j]) + ',' + str(t[i,j]) for (i,j) in A}
plt.figure(figsize=(12,4))
pos = {1 : (1,1), 2 : (1,2), 3 : (2,1), 4 : (2,2), 5 : (3,1), 6:(3,2), 7 : (4,1), 8 : (4,2)}
nx.draw_networkx_edge_labels(D, pos, edge_labels)
nx.draw_networkx(D, pos, labels= node_labels, node_color='cyan', node_size=1200)


Creamos ahora el objeto modelo y las variables de binarias de selección de arcos $x_{ij}$. Observar que las variables están indexadas por los conjuntos de arcos. Notar que las restricciones de ventanas de tiempo se ingresan como cotas de las variables.

In [ ]:
# Crear el objeto modelo
m = Model('shortest-path-time-windows')

# Crear las variables de selección de arcos
x = m.addVars(A, name="x", vtype=GRB.BINARY)

# Crear las variables de tiempo de visita en los nodos
# Las restricciones de ventanas de tiempo se representan como cotas de las variables
T = m.addVars(V, name="T", lb=a, ub=b)


Definimos la función objetivo a minimizar:

In [ ]:
m.setObjective(x.prod(c, '*'), GRB.MINIMIZE)

Definimos las restricciones del modelo:

1. Restricciones de grado

In [ ]:
# Arcos salientes de r
m.addConstr(x.sum(r,'*')  == 1, "grado_r")

# Arcos entrantes a s
m.addConstr(x.sum('*', s)  == 1, "grado_s")

# Balance de grados en los demas nodos
m.addConstrs(
    (x.sum('*',i) - x.sum(i,'*')  == 0 for i in Vi), "grado")

2. Restricciones de tiempo de visita en los nodos:

In [ ]:
# Tj >= Ti + tij si el arco (i,j) forma parte del camino:
m.addConstrs((T[j] >= T[i] + t[i,j] - M*(1 - x[i,j]) for (i,j) in A), "tiempos_visita")


Resolvemos el modelo y mostramos la solución. Notar que los valores de las variables pueden recuperarse también empleando el método `getAttr`:

In [ ]:
# Calcular la solucion optima
m.optimize()

# Escribir la solucion
if m.status == GRB.Status.OPTIMAL:
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)    
    vT = m.getAttr('x', T)
    print('T[{}]={}'.format(r, vT[r]))
    print('Arcos seleccionados:')
    for i,j in A:
        if vx[i,j] >= 0.99:
            print('{0} -> {1}; T[{1}]={2}'.format(i, j, vT[j] ))
    print("Costo: {}".format(m.objval))

Finalmente, grafiquemos la solución empleando `networkx` y `ipycytoscape`:

In [ ]:
import networkx as nx
import ipycytoscape
D = nx.DiGraph()
D.add_nodes_from(V)
for i in V:
    D.nodes[i]['etiq']= '{}:{}\n[{};{}]'.format(i, int(vT[i]), a[i], b[i])
D.add_edges_from(A)
for i,j in A:
    D.edges[i,j]['etiq'] = '({}, {})'.format(c[i,j], t[i,j])
    D.edges[i,j]['color'] =  '#9dbaea' if vx[i,j]<=0.1 else '#ff007f'
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=True)
grafo.set_style([{'selector': 'node', 'style' : {'background-color': '#11479e', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(etiq)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(etiq)'}}, 
                    {'selector': 'edge', 'style': {'width': 4, 'line-color': 'data(color)', 'font-size': '10px', 'label': 'data(etiq)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
grafo

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementacion de modelos de programacion lineal entera
# Problema de caminos mas cortos con ventanas de tiempo
# (shortest path with time windows)
# Luis M. Torres (EPN 2022)

from gurobipy import *

# Nodos y ventanas de tiempo
V , a, b = multidict({
    1 : (0, 1),
    2 : (2, 4),
    3 : (1, 3),
    4 : (4, 6),
    5 : (5, 6),
    6 : (5, 7),
    7 : (6, 7),
    8 : (7, 9),
})

# Arcos, costos y tiempos de tránsito 
A, c, t = multidict({
  (1, 2):  (3, 2), 
  (1, 3):  (20,1),
  (2, 3):  (10,2),
  (2, 4):  (5,2),
  (4, 3):  (2,3),
  (3, 5):  (2,3),
  (4, 5):  (5,1),
  (4, 6):  (10,1),
  (5, 6):  (2,3),
  (5, 7):  (-3,1),
  (6, 8):  (1, 2),  
  (7, 8):  (-3,1),  
  (8, 5):  (-3,1)})

# Nodo de salida
r = 1

# Nodo de llegada
s = 8

# Constante suficientemente grande:
M = quicksum([t[i,j] for (i,j) in A]) + quicksum(b[i] for i in V)

# --- los valores a partir de aqui se calculan automaticamente ---
# nodos internos: Vi := V \ {s, t}
Vi = tuplelist([i for i in V if i!=r and i!=s])
try:
    # Crear el objeto modelo
    m = Model('shortest-path-time-windows')

    # Crear las variables de selección de arcos
    x = m.addVars(A, name="x", vtype=GRB.BINARY)

    # Crear las variables de tiempo de visita en los nodos
    # Las restricciones de ventanas de tiempo se representan como cotas de las variables
    T = m.addVars(V, name="T", lb=a, ub=b)

    # Definir la funcion objetivo
    m.setObjective(x.prod(c, '*'), GRB.MINIMIZE)

    # Arcos salientes de r
    m.addConstr(x.sum(r,'*')  == 1, "grado_r")

    # Arcos entrantes a s
    m.addConstr(x.sum('*', s)  == 1, "grado_s")

    # Balance de grados en los demas nodos
    m.addConstrs(
        (x.sum('*',i) - x.sum(i,'*')  == 0 for i in Vi), "grado")

    # Tj >= Ti + tij si el arco (i,j) forma parte del camino:
    m.addConstrs((T[j] >= T[i] + t[i,j] - M*(1 - x[i,j]) for (i,j) in A), "tiempos_visita")

    # Calcular la solución óptima
    m.optimize()

    # Escribir la solucion
    if m.status == GRB.Status.OPTIMAL:
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)    
        vT = m.getAttr('x', T)
        print('T[{}]={}'.format(r, vT[r]))
        print('Arcos seleccionados:')
        for i,j in A:
            if vx[i,j] >= 0.99:
                print('{0} -> {1}; T[{1}]={2}'.format(i, j, vT[j] ))
        print("Costo: {}".format(m.objval))      
        
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')